## Data Cleaning and Normalizing Task

### Job description:
#### 1. Translate data from Vietnamese to English:

a. **kokotaru** 'Assert/cleaned_recipes_translated.txt'

b. **kitchenart** 'Assert/cleaned_recipes_2_translated.txt'


#### 2. Clean and Normalize data from 'txt' file to 'csv' file

   | Name of dish | Ingredient 1  | Ingredient 2  |...            |
   | -------------|---------------| --------------|---------------|
   | ...          |0/1            |0/1            |...            |


### Install the libraries

In [10]:
!pip install nltk

### Import shared libraries and functions

In [11]:
from Library_Used import *
from Shared_Functions import *

### Read data from files

**black_list.txt**, **units.txt**, **key_words.txt** are txt files containing lists of words that will be removed from data lines to filter out food ingredients.

- **black_list.txt** contains noise words to describe the properties and preparation methods of ingredients.

- **units.txt** contains words that are units of measurement and quantity of words for an ingredient.

- **key_words.txt** is similar to **black_list.txt**, a list of noise words for data, but instead of single words, key words will help you accurately identify clusters of noise words.

In [12]:
units = read_from_file("./Assert/units.txt","r_b_line")
key_words = read_from_file("./Assert/key_words.txt","r_b_line")
black_list = read_from_file("./Assert/black_list.txt","r_b_line")

### Set up tools to use the *nktl* library

- Download the necessary data (WordNet and OMW) to create **lemmatizer**.
- Create a **lemmatizer** to process words and make them singular.

In [13]:
nltk.download('wordnet')
nltk.download('omw-1.4')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\huyen\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\huyen\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


### Data Cleaning

In [14]:
def clean_ingredient(ingredient: str, units: list, key_words:list, black_list:list,lemmatizer):
    """
    Cleans a list of raw ingredients based on specified units, keywords, and blacklist
    Parameters:
        ingredient (str): a string which maybe contains one or more in it with description.
        units (list): contains words that are units of measurement, quantity words
        key_words (list): contains words that are signals to identify the noise part in the string
        black_list (list): noise words

    Returns:
        clean_parts(list): list containing one or more cleaned ingredients from the input string 'ingredient'
    """
    # Bước 1: Chuẩn hóa Unicode, đưa chuỗi về kí tự thường
    ingredient = unicodedata.normalize("NFKD", ingredient)
    ingredient = ingredient.lower()

    # Bước 2: thực hiện trước khi tách nguyên liệu: Loại bỏ nội dung trong ngoặc đơn
    ingredient = re.sub(r"\s*\(.*?\)", "", ingredient)
    
    # Bước 3: Tách thành các nguyên liệu khác nhau khi gặp ',', ';', 'and', 'or', 'with', 'in'
    parts = re.split(r",|;|\band\b|\bor\b|\bwith\b|\bin\b", ingredient, flags=re.IGNORECASE)
    clean_parts = []

    
    # Tiến thành duyệt qua từng phần đã tách được từ bước 3:
    for part in parts:
        # Bước 4.1: Loại bỏ kí tự không phải ASCII
        part = re.sub(r"[^\x00-\x7F]+", "", part)
        
        # Bước 4.2: Chuẩn hóa các phân số bị lỗi
        part = re.sub(r"\b\d+/[a-zA-Z]+\b", "", part)
        
        # Bước 4.3: Loại bỏ số lượng và đơn vị đo lường
        units_pattern = r"\b\d*[\d/]*\s*(" + "|".join(map(re.escape, units)) + r")\b"
        part = re.sub(units_pattern, "", part, flags=re.IGNORECASE)
        part = re.sub(r"\d+", "", part) 
        
        # Bước 4.4: Thay thế các ký tự '-', '/', '.', '*' thành khoảng trắng
        part = re.sub(r"[+/.*>]", " ", part)
        part = re.sub(r"(?<=\s)-|-(?=\s)|(?<=\d)-(?=\d)", " ", part)
        
        # Bước 4.5: Nếu có từ "of", chỉ giữ lại phần sau chữ "of"
        if " of " in part:
            part = part.split(" of ", 1)[-1]
        
        # Bước 4.6: Sử dụng các từ trong key_words để xác định cụm gây nhiễu và xóa chúng
        for word in key_words:
            pattern = rf"\b{re.escape(word)}\b.*"
            part = re.sub(pattern, "", part, flags=re.IGNORECASE)
        
        # Bước 4.7: Xóa các từ trong black_list mà có xuất hiện trong chuỗi. Chỉ xóa khi từ đó không gắn liền với dấu '-' khi vừa là tên vừa biểu thị đặc tính của nguyên liệu
        # (Ví dụ: all-purpose flour : bột mì đa dụng)
        for word in black_list:
            pattern = rf"(?<!-)\b{re.escape(word)}\b(?!-)" 
            part = re.sub(pattern, "", part, flags=re.IGNORECASE)
        
        # Bước 4.8: Xóa khoảng trắng thừa
        part = re.sub(r"\s+", " ", part)
        clean_part = part.strip()
        
        # Bước 4.9: Chuẩn hóa về dạng số ít của nguyên liệu (Ví dụ: apples -> apple)
        clean_part = lemmatizer.lemmatize(clean_part)

        # Bước 5: Trả về kết quả là một list chứa các nguyên liệu đã được làm sạch.
        if clean_part: 
            clean_parts.append(clean_part)
    
    return clean_parts

### Data Normalizing

In [15]:
def normalize_recipes_to_dataframe(file_paths, units, key_words, black_list,lemmatizer):
    """
    Processes one or more text files of recipes into a binary DataFrame of ingredients.

    Parameters:
        file_paths (list of str): List of paths to text files containing recipes.
        units (list): List of units to clean ingredients.
        key_words (list): List of keywords to clean ingredients.
        black_list (list): List of blacklisted words to remove from ingredients.

    Returns:
        pd.DataFrame: Binary DataFrame with ingredients as columns.
        list: List of unique ingredients.
    """
    combined_data = []

    # Đọc và xử lý từng file
    for file_path in file_paths:
        # Đọc nội dung file
        content = read_from_file(file_path, "r_b_str")

        # Tách từng món ăn dựa trên dòng gạch ngang
        dishes = content.split('-' * 50)
        dishes_with_no_ingredients =[]
        for dish in dishes:
            title_match = re.search(r"Title:\s*(.+)", dish)
            ingredients_match = re.search(r"Ingredients:\s*(.+)", dish, re.DOTALL)

            if title_match and ingredients_match:
                # Lấy tên món ăn
                title = title_match.group(1).strip()
                # Lấy phần nguyên liệu của món ăn
                ingredients_raw = ingredients_match.group(1).strip()
                ingredients_lines = ingredients_raw.splitlines() 

                if not ingredients_lines:  # Nếu danh sách nguyên liệu rỗng
                    combined_data.append({
                        'title': title,
                        'ingredients': []  # Hoặc ['No Ingredients'] nếu bạn muốn ghi chú
                    })
                    continue

                clean_ingredients = []
                for line in ingredients_lines:
                    # Bỏ qua các dòng chỉ chú thích về cách thực hiện...
                    if ':' in line or '=>' in line:
                        continue
                    # Làm sạch nguyên liệu
                    clean_ingredients.extend(clean_ingredient(line, units, key_words, black_list,lemmatizer))

                combined_data.append({
                    'title': title,
                    'ingredients': clean_ingredients
                })

    # Tạo DataFrame từ dữ liệu kết hợp
    df = pd.DataFrame(combined_data)

    # Lấy danh sách nguyên liệu phân biệt
    all_ingredients = sorted(set(ingredient.strip().lower()
                                 for ingredients in df["ingredients"]
                                 for ingredient in ingredients))

    # Tạo DataFrame nhị phân
    binary_df = pd.DataFrame(columns=["Name of dish"] + all_ingredients)
    binary_df["Name of dish"] = df["title"]  # Thêm cột tiêu đề món ăn

    for index, row in df.iterrows():
        ingredients_set = set(row["ingredients"])
        for ingredient in all_ingredients:
            binary_df.at[index, ingredient] = 1 if ingredient in ingredients_set else 0

    return binary_df, all_ingredients

In [16]:
file_paths = ["./Assert/cleaned_recipes_2_translated.txt"]

binary_df, all_ingredients = normalize_recipes_to_dataframe (file_paths,units,key_words,black_list,lemmatizer)

print(f"Number of dishes: {binary_df.shape[0]}")
print(f"Number of ingredients: {binary_df.shape[1]}")

binary_df.to_csv("../Assert/ingredients.csv", index=False, encoding="utf-8")
with open("./Testing/ingredient_list.txt",'w') as file:
    for item in all_ingredients:
        file.write(item + '\n')


Number of dishes: 682
Number of ingredients: 803
